In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import messagebox
import tkinter.font as tkFont
# Create a x and y size of a minesweeper game
x = np.arange(9)
y = np.arange(9)
MultiIndex = pd.MultiIndex.from_product([x, y], names=["x", "y"])
df = pd.DataFrame(index=MultiIndex)



In [2]:
# create a column with the number of mines 
#mines = np.random.choice([0 , 1], size=len(MultiIndex), p=[0.9,0.1])  # Generating random values for the mines
mines = 10  # Number of mines to place
mines_array = np.array([1] * mines + [0] * (len(MultiIndex) - mines))
np.random.shuffle(mines_array)  # Shuffle the array to randomize mine positions
df["mines"] = mines_array  # Adding a 'mine' column with random values of 0 or 1

In [3]:
# Define a function to calculate the sum of adjacent mines
def adjacent_sum(coord):
    cx, cy = coord
    # this if condition checks if the current cell is a mine. I add it from a next task
    if df.loc[(cx, cy), "mines"] == 1:
        return np.nan
    neighbors = df.loc[pd.IndexSlice[max(0, cx-1):min(x.max(), cx+1),
                           max(0, cy-1):min(y.max(), cy+1)], "mines"]
    return neighbors.sum()

df["adjacent"] = df.index.map(adjacent_sum).astype("Int64") # Applying the function to each index in the DataFrame df["adjacent"] = [adjacent_sum(idx) for idx in df.index]
df["adjacent"].unstack()

y,0,1,2,3,4,5,6,7,8
x,,,,,,,,,
0,1,1,0,1,2,3,<NA>,1,0
1,<NA>,2,1,2,<NA>,<NA>,2,1,0
2,2,3,<NA>,2,2,2,2,1,1
3,1,<NA>,2,1,0,0,1,<NA>,2
4,1,1,1,0,0,0,1,2,<NA>
5,0,0,1,1,1,0,0,1,1
6,0,0,1,<NA>,2,1,1,0,0
7,0,0,1,1,2,<NA>,1,0,0
8,0,0,0,0,1,1,1,0,0


In [4]:
"""
root = tk.Tk()
root.title(f"Minesweeper {max(x)+1}x{max(y)+1}")

buttons = {}
font_style = tkFont.Font(family="Arial", size=16, weight="bold")
colors = {
    1: "blue",
    2: "green",
    3: "red",
    4: "darkblue",
    5: "brown",
    6: "cyan",
    7: "black",
    8: "gray"
}
mines_left = mines  
flag_label = tk.Label(root, text=f"Flags left: {mines_left}", font=font_style)
flag_label.grid(row=0, column=len(y), columnspan=2, sticky="w")
"""
def update_flag_count(delta):
    global mines_left
    mines_left += delta
    flag_label.config(text=f"Flags left: {mines_left}")

def check_win():
    for (cx, cy) in buttons:
        if df.loc[(cx, cy), "mines"] == 0 and buttons[(cx, cy)]["state"] != "disabled":
            return False
    return True

def on_click(cx, cy):
    btn = buttons[(cx, cy)]
    if btn["text"] == "🚩":
        return
    if df.loc[(cx, cy), "mines"] == 1:
        # show all mines and end the game
        for (i, j), btn in buttons.items():
            if df.loc[(i, j), "mines"] == 1:
                # if the cell was flagged, change the color to green, otherwise show a mine
                if btn["text"] == "🚩":  
                    btn.config(bg="green") 
                else:
                    btn.config(text="💣", bg="red")
            # if the cell was flagged but it was not a mine, change the color to yellow
            else:
                if btn["text"] == "🚩":
                    btn.config(bg ="yellow")
        messagebox.showinfo("Game Over", "💣 You hit a mine!")
        root.destroy()
    def reveal(cx, cy):
        btn = buttons[(cx, cy)]
        # if the cell is already revealed or disabled, do nothing
        if btn["state"] == "disabled" or btn["text"] == "🚩":
            return
        adj = df.loc[(cx, cy), "adjacent"]
        # if the cell is 0 we don`t show 0 on interface`
        #text = "" if adj == 0 else str(adj)
        #fg_color = colors.get(adj, "black") if adj > 0 else "black"
    
        #buttons[(cx, cy)].config(text=text, state="disabled", bg="lightgrey", fg ="red")
        buttons[(cx, cy)].config(state="disabled", bg="lightgrey")
        if adj > 0:
            lbl = tk.Label(root, text=str(adj),
                           fg=colors.get(adj, "black"),
                           bg="lightgrey",font=font_style)
            lbl.grid(row=cx, column=cy)
    

        if adj == 0:
                for nx in range(max(0, cx-1), min(df.index.get_level_values("x").max()+1, cx+2)):
                    for ny in range(max(0, cy-1), min(df.index.get_level_values("y").max()+1, cy+2)):
                        if (nx, ny) != (cx, cy):
                            reveal(nx, ny)
        if check_win():
            messagebox.showinfo("Victory!", "🎉 You cleared the field!")
            root.destroy()
    reveal(cx,cy)


def on_right_click(event, cx, cy):
    btn = buttons[(cx, cy)]
    # if the cell is opened or disabled, do nothing
    if btn["state"] == "disabled":
        return
    # change the flag
    if btn["text"] == "🚩":
        btn.config(text="")  # delete the flag
        update_flag_count(1)
    else:
        btn.config(text="🚩",fg = "red")  # create the іflag
        update_flag_count(-1)
"""
for i in range(len(x)):
    for j in range(len(y)):
        b = tk.Button(root, width=3, height=1, font=font_style,
                      command=lambda cx=i, cy=j: on_click(cx, cy)) #
        b.grid(row=i, column=j)
        b.bind("<Button-3>", lambda event, cx=i, cy=j: on_right_click(event, cx, cy))
        buttons[(i, j)] = b # Create a button for each cell and save there coordinates (i, j) as a key in the buttons dictionary anb "b" as a value

root.mainloop()
"""
def restart_game():
    root.destroy()
    main()

def main():
    global root, buttons, mines_left, flag_label, colors, font_style
    root = tk.Tk()
    root.title(f"Minesweeper {max(x)+1}x{max(y)+1}")

    buttons = {}
    font_style = tkFont.Font(family="Arial", size=16, weight="bold")
    colors = {
        1: "blue",
        2: "green",
        3: "red",
        4: "darkblue",
        5: "brown",
        6: "cyan",
        7: "black",
        8: "gray"
    }
    mines_left = mines 
    """
    flag_label = tk.Label(root, text=f"Flags left: {mines_left}", font=font_style)
    flag_label.grid(row=0, column=len(y), columnspan=2, sticky="w")
    restart_btn = tk.Button(root, text="Restart", font=font_style, bg="orange", command=restart_game)
    restart_btn.grid(row=0, column=len(y)+2, columnspan=2, sticky="w")
    """
    restart_btn = tk.Button(root, text="Restart", font=font_style, bg="orange", command=restart_game)
    restart_btn.grid(row=0, column=len(y), columnspan=2, sticky="n")

    # Лічильник прапорців – нижче
    flag_label = tk.Label(root, text=f"Flags left: {mines_left}", font=font_style)
    flag_label.grid(row=1, column=len(y), columnspan=2, sticky="w")
    for i in range(len(x)):
        for j in range(len(y)):
            b = tk.Button(root, width=3, height=1, font=font_style,
                        command=lambda cx=i, cy=j: on_click(cx, cy)) #
            b.grid(row=i, column=j)
            b.bind("<Button-3>", lambda event, cx=i, cy=j: on_right_click(event, cx, cy))
            buttons[(i, j)] = b # Create a button for each cell and save there coordinates (i, j) as a key in the buttons dictionary anb "b" as a value

    root.mainloop()

In [5]:
main()